# **Question 1**

In [3]:
import requests
# HTTP library for making web requests to download pages
#Purpose: HTTP library for making web requests
#Function: Downloads web page content from URLs

In [2]:
from bs4 import BeautifulSoup
# HTML parser to extract data from web pages
#Purpose: HTML/XML parsing library
#Function: Converts raw HTML into a navigable tree structure

In [4]:
import pandas as pd
# Data manipulation library for organizing scraped data
#Purpose: Data manipulation and analysis library
#Function: Handles structured data operations

In [5]:
import time
# For adding delays between requests (respectful scraping)

In [6]:
import re
# Regular expressions for text pattern matching and cleaning

In [7]:
from urllib.parse import urljoin, urlparse
# URL manipulation utilities

In [8]:
class BooksScraper:
    def __init__(self, base_url="https://books.toscrape.com/"):
        self.base_url = base_url
        self.session = requests.Session()
        # Add headers to mimic a real browser
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        self.books_data = []

    def get_star_rating(self, star_class):
        """Convert star rating class to number of stars"""
        rating_map = {
            'One': 1,
            'Two': 2,
            'Three': 3,
            'Four': 4,
            'Five': 5
        }

        # Extract the rating from class like "star-rating Three"
        for rating_word, rating_num in rating_map.items():
            if rating_word in star_class:
                return rating_num
        return 0

    def clean_price(self, price_text):
        """Clean price text and convert to float"""
        # Remove currency symbol and extra spaces
        price_cleaned = re.sub(r'[£$€]', '', price_text.strip())
        try:
            return float(price_cleaned)
        except ValueError:
            return 0.0

    def scrape_book_details(self, book_element):
        """Extract details from a single book element"""
        try:
            # Title
            title_element = book_element.find('h3').find('a')
            title = title_element.get('title') if title_element else "N/A"

            # Price
            price_element = book_element.find('p', class_='price_color')
            price_text = price_element.text if price_element else "£0.00"
            price = self.clean_price(price_text)

            # Availability
            availability_element = book_element.find('p', class_='instock availability')
            availability_text = availability_element.text.strip() if availability_element else "Out of stock"
            availability = "In stock" if "In stock" in availability_text else "Out of stock"

            # Star Rating
            star_element = book_element.find('p', class_=re.compile(r'star-rating'))
            if star_element:
                star_classes = star_element.get('class', [])
                star_rating_class = ' '.join(star_classes)
                star_rating = self.get_star_rating(star_rating_class)
            else:
                star_rating = 0

            return {
                'Title': title,
                'Price': price,
                'Availability': availability,
                'Star_Rating': star_rating
            }

        except Exception as e:
            print(f"Error extracting book details: {e}")
            return None

    def scrape_page(self, url):
        """Scrape books from a single page"""
        try:
            print(f"Scraping: {url}")
            response = self.session.get(url, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all book containers
            books = soup.find_all('article', class_='product_pod')

            page_books = []
            for book in books:
                book_data = self.scrape_book_details(book)
                if book_data:
                    page_books.append(book_data)

            print(f"Found {len(page_books)} books on this page")
            return page_books, soup

        except requests.RequestException as e:
            print(f"Error fetching page {url}: {e}")
            return [], None
        except Exception as e:
            print(f"Error parsing page {url}: {e}")
            return [], None

    def find_next_page_url(self, soup, current_url):
        """Find the URL of the next page"""
        if not soup:
            return None

        # Look for next page link
        next_link = soup.find('li', class_='next')
        if next_link:
            next_a = next_link.find('a')
            if next_a and next_a.get('href'):
                # Construct absolute URL
                next_relative_url = next_a.get('href')
                next_url = urljoin(current_url, next_relative_url)
                return next_url

        return None

    def scrape_all_books(self):
        """Scrape books from all pages"""
        print("Starting to scrape all books from Books to Scrape...")

        current_url = self.base_url
        page_number = 1

        while current_url:
            print(f"\n--- Page {page_number} ---")

            # Scrape current page
            page_books, soup = self.scrape_page(current_url)

            if not page_books:
                print("No books found on this page, stopping...")
                break

            # Add books to our collection
            self.books_data.extend(page_books)

            # Find next page URL
            next_url = self.find_next_page_url(soup, current_url)

            if next_url:
                current_url = next_url
                page_number += 1
                # Be respectful with delays
                time.sleep(1)
            else:
                print("No more pages found.")
                break

        print(f"\nScraping completed! Total books collected: {len(self.books_data)}")
        return self.books_data

    def save_to_csv(self, filename="books.csv"):
        """Save scraped data to CSV file"""
        if not self.books_data:
            print("No data to save!")
            return

        # Create DataFrame
        df = pd.DataFrame(self.books_data)

        # Save to CSV
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")

        # Display summary statistics
        print(f"\n--- Data Summary ---")
        print(f"Total books: {len(df)}")
        print(f"Average price: £{df['Price'].mean():.2f}")
        print(f"Price range: £{df['Price'].min():.2f} - £{df['Price'].max():.2f}")
        print(f"\nAvailability distribution:")
        print(df['Availability'].value_counts())
        print(f"\nStar rating distribution:")
        print(df['Star_Rating'].value_counts().sort_index())

        return df

    def display_sample_data(self, num_samples=5):
        """Display sample of scraped data"""
        if not self.books_data:
            print("No data available!")
            return

        df = pd.DataFrame(self.books_data)
        print(f"\n--- Sample Data (first {num_samples} books) ---")
        print(df.head(num_samples).to_string(index=False))

In [11]:
def main():
    try:
        print("=" * 60)
        print("BOOKS TO SCRAPE - WEB SCRAPER")
        print("=" * 60)

        scraper = BooksScraper()

        # DATA COLLECTION PHASE
        print("Phase 1: Scraping all books from website...")
        books_data = scraper.scrape_all_books()

        # DATA PROCESSING AND EXPORT PHASE
        if books_data:
            print("\nPhase 2: Processing and analyzing data...")

            scraper.display_sample_data()

            df = scraper.save_to_csv("books.csv")

            # ADVANCED ANALYSIS - Additional insights
            print(f"\n--- Top 10 Most Expensive Books ---")
            if len(books_data) > 0:
                # NLARGEST METHOD - Finds highest values in specified column
                df_sorted = df.nlargest(10, 'Price')
                # COLUMN SELECTION - Display only relevant columns
                print(df_sorted[['Title', 'Price', 'Star_Rating']].to_string(index=False))

            # SUCCESS MESSAGE
            print("\n" + "=" * 60)
            print("SCRAPING COMPLETED SUCCESSFULLY!")
            print(f"Total books scraped: {len(books_data)}")
            print("Data saved to: books.csv")
            print("=" * 60)

        else:
            # FAILURE HANDLING
            print("ERROR: No books were scraped!")
            print("Please check:")
            print("1. Internet connection")
            print("2. Website accessibility")
            print("3. HTML structure changes")

    except KeyboardInterrupt:
        # GRACEFUL SHUTDOWN - Handle Ctrl+C interruption
        print("\nScraping interrupted by user.")
    except Exception as e:
        # UNEXPECTED ERROR HANDLING
        print(f"Unexpected error occurred: {e}")
        print("Please check your internet connection and try again.")

In [12]:
if __name__ == "__main__":
    main()

BOOKS TO SCRAPE - WEB SCRAPER
Phase 1: Scraping all books from website...
Starting to scrape all books from Books to Scrape...

--- Page 1 ---
Scraping: https://books.toscrape.com/
Found 20 books on this page

--- Page 2 ---
Scraping: https://books.toscrape.com/catalogue/page-2.html
Found 20 books on this page

--- Page 3 ---
Scraping: https://books.toscrape.com/catalogue/page-3.html
Found 20 books on this page

--- Page 4 ---
Scraping: https://books.toscrape.com/catalogue/page-4.html
Found 20 books on this page

--- Page 5 ---
Scraping: https://books.toscrape.com/catalogue/page-5.html
Found 20 books on this page

--- Page 6 ---
Scraping: https://books.toscrape.com/catalogue/page-6.html
Found 20 books on this page

--- Page 7 ---
Scraping: https://books.toscrape.com/catalogue/page-7.html
Found 20 books on this page

--- Page 8 ---
Scraping: https://books.toscrape.com/catalogue/page-8.html
Found 20 books on this page

--- Page 9 ---
Scraping: https://books.toscrape.com/catalogue/page-9.